<a href="https://colab.research.google.com/github/2025ab05031/Sem1_ML_Assignment2/blob/main/2025ab05031_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2025 sem1 ML Assignment 2
Name: B. Krishna Mohan
BITS ID: 2025AB05031
---



In [ ]:
print("ML Assignment 2")

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
#from ml_core import load_data, train_test, train_and_evaluate, evaluate_all

st.title("Classification Playground")

data_file = st.sidebar.file_uploader("Upload CSV", type=["csv"])
if data_file is not None:
    X, y = load_data(data_file)
    X_train, X_test, y_train, y_test = train_test(X, y)

    mode = st.sidebar.radio("Mode", ["Single model", "Compare all"])

    # collect params ...

    if st.button("Train"):
        if mode == "Single model":
            model, y_pred, y_score, metrics = train_and_evaluate(
                selected_model, params.get(selected_model, {}),
                X_train, X_test, y_train, y_test
            )
            st.write(metrics)
            # draw confusion matrix / ROC
        else:
            df_results = evaluate_all(X_train, X_test, y_train, y_test,
                                      global_params=params)
            st.dataframe(df_results)
            st.bar_chart(df_results.set_index("Model")["accuracy"])


In [ ]:
!pip install -q streamlit
!npm install -q localtunnel
!pip install Flask

In [ ]:
# 1. Install necessary packages

# 2. Create the Streamlit application file (using a magic command)
%%writefile app.py
import streamlit as st
st.title("My Streamlit App in Colab")
st.write("Hello, World!")

# 3. Run the Streamlit app and expose it via localtunnel



In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501